In [129]:
import numpy as np
from IPython.display import clear_output
from random import randrange, choice
class Game():
    def __init__(self,colum,fil):
        self.mesa= np.zeros( (fil,colum ) )
        self.columnas=colum-1
        self.filas=fil-1
        self.end=0
        self.mov=0
        
    def movimiento(self,colum,valor):
        x=self.filas
        y=colum
        celda=self.mesa[x-1][y]
        while(celda==0 and (x-1)>=0):
            x=x-1
            celda=self.mesa[x-1][y]
        self.mesa[x][y]=valor
        return x
    
    def verifica_multiplos(self,x,y):
        valor= self.mesa[x][y]
        suma=valor
        #Celda de arriba
        if(x-1>=0):
            if(self.mesa[x-1][y]==valor):
                suma=suma+self.mesa[x-1][y]
                self.mesa[x-1][y]=0.0
        #Celda de Abajo
        if(x+1<=self.filas):
            if(self.mesa[x+1][y]==valor):
                suma=suma+self.mesa[x+1][y]
                self.mesa[x+1][y]=0.0
        #Celda izquierda
        if(y-1>=0):
            if(self.mesa[x][y-1]==valor):
                suma=suma+self.mesa[x][y-1]
                self.mesa[x][y-1]=0.0
        #Celda derecha
        if(y+1<=self.columnas):
            if(self.mesa[x][y+1]==valor):
                suma=suma+self.mesa[x][y+1]
                self.mesa[x][y+1]=0.0
        if(valor!=suma):
            self.mesa[x][y]=suma
            self.verifica_multiplos(x,y)
            
        self.verificar_espacios()
        
    def verificar_espacios(self):
        for x in range(self.filas+1):
            for y in range(self.columnas+1):
                if(x+1<=self.filas):
                    if(self.mesa[x][y]==0.0 and self.mesa[x+1][y]!=0.0):
                        self.mesa[x][y]=self.mesa[x+1][y]
                        self.mesa[x+1][y]=0.0
                        self.verifica_multiplos(x,y)
        self.verificar_end()
    
    def verificar_end(self):
        for y in range(self.columnas+1):
            if(self.mesa[self.filas][y]!=0.0):
                self.end=1
                
    def run(self):
        
        while(self.end==0):
            clear_output()
            print(self.mesa)
            valor= choice([2.0,4.0,8.0,16.0,32.0])
            print(valor)
            y=int(input())
            x= self.movimiento(y,valor)
            self.verifica_multiplos(x,y)
        print(self.mesa)
                
       
    
    
    

In [131]:
game= Game(3,4)
game.run()

[[  2.  32.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
16.0


KeyboardInterrupt: 